<a href="https://colab.research.google.com/github/MeidanGR/SER_DeepLearning_LSTM/blob/main/EmotionRecognition_DL_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Speech Emotion Recognition algorithm**
*Using Deep Learning (LSTM) model*

B.Sc. Final project by Meidan Greenberg; Linoy Hadad;

Instructor: Dr. Dima Alberg

# UNDER DEVELOPMENT.

# **PACKAGES & GOOGLE AUTH**

In [1]:
%%capture
!pip install soundfile
!pip install noisereduce
!pip install pydub
!pip install pywt

In [2]:
import numpy as np
import librosa
import pywt
import noisereduce as nr
import IPython.display as ipd #Audio player
import os
import pandas as pd
import sklearn
from pydub import AudioSegment, effects


/usr/local/lib/python3.7/dist-packages/noisereduce/noisereduce.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [6]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


# **LOADING FULL DATA**
The speech emotion audio databases used:

- RAVDASS: https://zenodo.org/record/1188976#.X4sE0tDXKUl
  - 1440 files = 24 actors x 60 trails per actor
  - 8 Emotions (neutral, calm, happy, sad, angry, fearful, disgust, surprised).
-TESS: https://tspace.library.utoronto.ca/handle/1807/24487
  - 2800 files = 2 actors x 200 pharses x 7 emotions
  - 7 Emotions (neutral, happiness, sadness, anger, fear, disgust, pleasant surprise)
   - ('calm' is not a part of this DB).


## **RAVDESS Database**

All of  RAVDESS files has a unique filename. The filename consists of a 7-part numerical identifier (e.g., 02-01-06-01-02-01-12.mp4). These identifiers define the stimulus characteristics: 

### Filename identifiers 

Modality (01 = full-AV, 02 = video-only, **03 = audio-only**).

Vocal channel (**01 = speech**, 02 = song).

**Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).**

Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the 'neutral' emotion.

Statement (01 = "Kids are talking by the door", 02 = "Dogs are sitting by the door").

Repetition (01 = 1st repetition, 02 = 2nd repetition).

Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).

Only files with the format: 03-01-XX-XX-XX-XX-XX.Wav has been imported
 **(speech audio only)**
 
## **TESS Database**

The TESS Database file name contain the emotion by text, for e.g. "YAF_youth_happy.wav". Therefore a find_emotion function has been executed.



---

# **FEATURE EXTRACTION**
...


In [124]:
#Normalization test for one sample.
#

print('Original: librosa.load')
x,sr = librosa.load('/content/drive/My Drive/AudioFiles/RAVDESS/Actor_16/03-01-08-02-01-01-16.wav', sr = None, duration = None)
print(np.shape(x))
ipd.display(ipd.Audio(data = x, rate=sr))

from pydub import AudioSegment, effects  
rawsound = AudioSegment.from_file('/content/drive/My Drive/AudioFiles/RAVDESS/Actor_16/03-01-08-02-01-01-16.wav', "wav",duration = None) 
normalizedsound = effects.normalize(rawsound, headroom = 0)  

print('Normalized: AudioSegment.from_file')
samples = normalizedsound.get_array_of_samples()
print(np.shape(samples))
normalizedsound

#ipd.display(ipd.Audio(data = rawsound, rate=sr))
ipd.display(ipd.Audio(data = samples, rate=sr))


Original
(169770,)
0.0076293945


Normalized
(169770,)
1695


In [11]:
#Emotion kind validation function for TESS database, due to the emotions are written within the file names.
def find_emotion(name): 
        if('neutral' in name): return "01"
        elif('happy' in name): return "03"
        elif('sad' in name): return "04"
        elif('angry' in name): return "05"
        elif('fear' in name): return "06"
        elif('disgust' in name): return "07"
        elif('ps' in name): return "08"
        else: return "-1"

#Initizlizing data lists
audio_data = []
sample_rate = []
mfcc = []
zcr = []
rms = []
dwt = []
pitch = []
emotions = []

#Running over BOTH databases for frames & features extraction (x)
folder_path = '/content/drive/My Drive/AudioFiles'


for subdir, dirs, files in os.walk(folder_path):
  for file in files:
    try:  

  # Loading file frames & normalizing to -3 dBFS
      rawsound = AudioSegment.from_file(os.path.join(subdir,file), "wav",duration = None) 
      normalizedsound = effects.normalize(rawsound, headroom = -3) #-3 dBFS
      normal_x = np.array(normalizedsound.get_array_of_samples(), dtype = float)

      _, sr = librosa.load(path = os.path.join(subdir,file), sr = None) # sr (the sample rate) is used for librosa's features extraction, _ is irrelevant.

  # Features extraction
      f1 = librosa.feature.mfcc(normal_x, sr=sr, S=None, n_mfcc=13,dct_type=2, norm='ortho', lifter=0) # MFCC
      f2 = librosa.feature.zero_crossing_rate(normal_x , frame_length=2048, hop_length=512,center=True) # ZCR
      f3 = librosa.feature.rms(normal_x , S=None, frame_length=2048, hop_length=512, center=True, pad_mode='reflect') # Energy - Root Mean Square
      cA, cD = pywt.dwt(normal_x, 'db2', 'sym') # DWT
      f0 = librosa.yin(normal_x, fmin=20, fmax = 20000, sr = sr, frame_length=2048, win_length=None, hop_length=None) # Pitch

  # Emotion extraction (y)      
      if (find_emotion(file) != "-1"): #TESS database validation
        name = find_emotion(file)
      else: 
        name = file[6:8]               #RAVDESS database validation

  # Filling the data lists for each iteration (each file)
      audio_data.append(x.T) 
      sample_rate.append(sr) 
      mfcc.append(f1)
      zcr.append(f2)
      rms.append(f3)
      dwt.append(cD)
      pitch.append(f0)
      emotions.append(name)  


    except ValueError:
      continue

# **NORMALIZING AUDIO DATA**
& Creating a DataFrame

In [12]:
df = pd.DataFrame(columns= ['x','sample_rate', 'mfccs', 'zcr', 'rms', 'dwt', 'pitch', 'y']) #DataFrame is used to 1) Visualization, 
#                                                                          2) Converting emotion number into a name using 'map' function.
#normal_array = []

"""
#Normalizing the audio_data arrays.
from pydub import AudioSegment


def match_target_amplitude(sound, target_dBFS):
    change_in_dBFS = target_dBFS - sound.dBFS
    return sound.apply_gain(change_in_dBFS)

folder_path = '/content/drive/My Drive/AudioFiles/RAVDESS/Actor_16/03-01-08-02-01-01-16.wav'

def normalize(x, axis=0):
    return sklearn.preprocessing.minmax_scale(x, axis=axis)

x = librosa.load(folder_path)
print(normalize(x))

for i in audio_data:
  sound = AudioSegment.from_numpy_array(i,48000)
  normal_array.append(match_target_amplitude(sound, -3.0)) 
  #norm = np.linalg.norm(i)
  #normal_array.append(np.array(i/norm))
"""
df.x = audio_data
df.sample_rate = sample_rate
df.mfccs = mfcc
df.zcr = zcr
df.rms = rms
df.dwt = dwt
df.pitch = pitch

df.y = emotions
df.y = df.y.map({'01' : 'neutral', '02' : 'calm', '03' : 'happy', '04' : 'sad', '05' : 'angry',
                             '06' : 'fearful', '07' : 'disgust', '08' : 'suprised'})

print(df.info())
df.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4238 entries, 0 to 4237
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   x            4238 non-null   object
 1   sample_rate  4238 non-null   int64 
 2   mfccs        4238 non-null   object
 3   zcr          4238 non-null   object
 4   rms          4238 non-null   object
 5   dwt          4238 non-null   object
 6   pitch        4238 non-null   object
 7   y            4238 non-null   object
dtypes: int64(1), object(7)
memory usage: 265.0+ KB
None


,x,sample_rate,mfccs,zcr,rms,dwt,pitch,y
4214,"[0.00046939997, 7.246247e-05, -0.0005050608, -...",48000,"[[503.4134696874334, 503.4134696874334, 503.41...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[24000.0, 24000.0, 24000.0, 24000.0, 24000.0, ...",fearful
2487,"[0.00046939997, 7.246247e-05, -0.0005050608, -...",24414,"[[634.8322045647006, 714.2743724531908, 789.94...","[[0.0322265625, 0.1494140625, 0.27294921875, 0...","[[831.6262441511436, 973.2094656663077, 1578.4...","[82.67027881893226, -0.37960445908252183, 28.5...","[4078.7285367768854, 4051.4589689988943, 4085....",fearful
955,"[0.00046939997, 7.246247e-05, -0.0005050608, -...",24414,"[[566.1417207784651, 569.3959271511982, 581.05...","[[0.04736328125, 0.0517578125, 0.072265625, 0....","[[131.93627131119024, 162.21881471535985, 175....","[47.15267754857617, -83.2151233890901, -12.466...","[26.490947719761973, 1641.843265478427, 1644.1...",neutral
436,"[0.00046939997, 7.246247e-05, -0.0005050608, -...",24414,"[[703.2343887518401, 780.9063181456484, 801.28...","[[0.236328125, 0.40771484375, 0.60107421875, 0...","[[946.4283589748751, 1390.8988122287599, 1714....","[16.53405576378645, -10.075360387616772, 38.87...","[7477.7506731394715, 6572.866387623534, 7461.2...",angry
2328,"[0.00046939997, 7.246247e-05, -0.0005050608, -...",24414,"[[561.3486832212068, 620.5277778838805, 697.60...","[[0.0087890625, 0.06982421875, 0.16162109375, ...","[[476.14468210910564, 515.3800407716438, 753.6...","[-5.5113519212621505, 2.812334319067221, -4.47...","[4069.415191526426, 3651.5337510201457, 4028.3...",disgust


# AUDIO & EMOTION CHECKS

In [ ]:
#Enter a row num between [0-4239]
row = 100

print('Emotion:', df.y[row])
ipd.display(ipd.Audio(data = df.x[row], rate=df.sample_rate[row]))


# **TRAIN & TEST SETS SPLIT**

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.x, df.y, test_size = 0.3, random_state = 42)
